# OneHot Encoding
Esta notebook está pensada para estudiar una tarea muy común en los proyectos de ML que es la codificación de datos categóricos utilizando la clase `OneHotencoding` de `sklearn`. <br>
Parte de esta notebook está basada en el Tutorial de [Data School](https://www.youtube.com/channel/UCnVzApLJE2ljPZSeQylSEyg) que se puede encontrar [acá](https://www.youtube.com/watch?v=irHhDMbw3xo)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.shape

(891, 12)

In [6]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
# por el momento solo nos vamos a quedar con algunas columnas
df = df.loc[:,['Survived', 'Pclass', 'Sex', 'Embarked']]

In [13]:
# eliminamos las ds filas que tienen valores faltantes en la columna Embarked
df.dropna(subset='Embarked', inplace=True)
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [14]:
df

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S
...,...,...,...,...
886,0,2,male,S
887,1,1,female,S
888,0,3,female,S
889,1,1,male,C


In [16]:
# tenemos dos features numericas y dos features categóricas
df.dtypes

Survived     int64
Pclass       int64
Sex         object
Embarked    object
dtype: object

## `LogisticRegression`

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

### Feature = `Pclass`


Vamos a empezar entrenando un modelo muy sencillo solo con la variable Pclass y ver qué obtenemos

In [20]:
# vamos a empezar con un modelo muy sencillo en el que solo vamos a usar la variable Pclass para entrenar un modelo de 
# regressin logistica
X = df.loc[:, ['Pclass']]
y = df.Survived
X.shape, y.shape

((889, 1), (889,))

In [25]:
# 1. instanciamos el modelo de regresion lineal
logreg = LogisticRegression(solver='lbfgs')

# Vamos a utilizar Validación cruzada
# 2. Entrenamos el modelo utilizando cross_val
cross_val_score(logreg, X, y, cv=5, scoring='accuracy')


array([0.6011236 , 0.6741573 , 0.6741573 , 0.71910112, 0.72316384])

In [26]:
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

Utilizando un modelo de regresión logística y una sola variable obtuvimos un accuracy del 67 %.<br>
Es posible mejorar este modelo?<br>

Una manera de empezar a mejorar el modelo, sin necesidad de ajustar hiperparámetros, es agregar más variables al modelo.

### Features = `Pclass`, `Sex`, `Embarked`


Para poder incorporar a nuestro modelo las variables categóricas vamos a tener que codificarlas primero.<br>
Para hacer esto se puede usar Pandas o Sklear. En este caso vamos a utiliar sklearn.

In [27]:
from sklearn.preprocessing import OneHotEncoder

In [29]:
# redefinimos la variable X
X = df[['Pclass', 'Sex', 'Embarked']]
X

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S
...,...,...,...
886,2,male,S
887,1,female,S
888,3,female,S
889,1,male,C


#### `make_column_transformer`

`make_column_transformer` es una clase de sklearn que se utilizar cuando se tiene un DF y se quieren aplicar prepocesamientos diferentes a diferentes columnas. En este caso tenemos dos columnas a las que queremos codificar con OneHot y una columna a la que no le queremos hacer nada.<br>
Notar que esta clase también la podría utiliar si quisiera imputar los valores faltantes de algunas columnas


In [33]:
from sklearn.compose import make_column_transformer

# 1. Instanciamos la clase make_column_transformer
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),   # aplicamos OneHot a ests columnas
    remainder='passthrough'                  # al resto de las columnas (las "remainder"), las pasmos de largo
)

# 2. fit_transform
column_trans.fit_transform(X)
#nota: en realidad no vamos a necesitar ahora la DF transformada, acá está hecha solo para ver como quedaría.
# en la siguiente sección vamos a utilizar `make_pipeline` y por ende la transformación se a relizar allí

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

#### `make_pipeline`

In [35]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(column_trans,    # pasamos la transformacin instanciada
                     logreg)          # pasamos el modelo instanciado



**Importante:** Tal como estamos desarrolando esta estrategia, NO es necesario tener los datos codificados antes de correr la pipeline. La pipeline tiene dentro una etapa de codificación.

In [39]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy')

array([0.76404494, 0.79213483, 0.76966292, 0.75280899, 0.78531073])

In [40]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [43]:
# Finalmente entrenamos el modelo usando la Pipeline
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

### Making Predictions

Si bien esto que vamos a hacer NO esta bien desde el punto de vista de ML, lo vamos a hacer para entender cómo tendríamos que hacer en caso de tener nuevaos datos que quisieramos evaluar

In [41]:
# Creamos datos "nuevos", usando los mismos datos que teníamos antes. Esto es solo a fines de entender como predecir usando la función pipe,
# pero el resultado numérico no tiene valor real.
X_new = X.sample(5, random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


Notar que en este caso no tenemos un modelo que poder usar haciendo model.predict(X_new).<br>
En este caso tenemos una pipeline que tiene un modelo, pero no tenemos un modelo separado. <br>
Por lo tando vamos a tener que hacer lo siguiente...

In [44]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)